In [51]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import math
import sklearn
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import sklearn.datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import optuna 
from sklearn.metrics import mean_squared_error
from IPython.display import clear_output

from datetime import datetime

In [23]:
train = pd.read_csv('..\\kaggle_data\\train.csv')
test = pd.read_csv('..\\kaggle_data\\test.csv')

In [24]:
cat_features = [
    "cat0", "cat1", "cat2", "cat3", "cat4", "cat5", "cat6", "cat7", 
    "cat8", "cat9"
]

cont_features = [
    "cont0", "cont1", "cont2", "cont3", "cont4",
    "cont5", "cont6", "cont7", "cont8", "cont9", "cont10", 
    "cont11", "cont12", "cont13"
]
alpha = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z' ]
alpha_conversion = {}
for index in range(0, len(alpha)):
    alpha_conversion[alpha[index]] = index / 29
alpha_conversion

for cat in cat_features:
    train[cat].replace(alpha_conversion, inplace=True)
    train[cat] = train[cat].astype('float')
train = train.drop(['id'], axis=1)

for cat in cat_features:
    test[cat].replace(alpha_conversion, inplace=True)
    test[cat] = test[cat].astype('float')

In [25]:
columns = [col for col in train.columns.to_list() if col not in ['id','target']]
data=train[columns]
target=train['target']
test_x = test.drop(['id'], axis=1)

In [80]:
preds = []
gbm = None
def objective(trial):
    global preds
    global gbm
    data, target = sklearn.datasets.load_breast_cancer(return_X_y=True)
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.25, random_state=42)
    d_train = lgb.Dataset(data=train_x, label=train_y)
    d_valid = lgb.Dataset(data=valid_x, label=valid_y, reference=d_train)

    param = {
        "metric": "rmse",
        "verbosity": -1,
        "boosting_type": "gbdt",
        
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-10, 100.0, log = True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-10, 100.0, log = True),
        
        "num_leaves": trial.suggest_int("num_leaves", 2, 1024, step = 1),
        
        "feature_fraction": trial.suggest_float("feature_fraction", 0.01, 1.0, step = 0.01),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0, step = 0.01),
        
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 14, step = 1),
        
        "min_child_samples": trial.suggest_int("min_child_samples", 2, 1000, step = 1),
    }

    param['early_stopping_round'] = 100
    
    gbm = lgb.train(param, train_set=d_train, valid_sets=d_valid)
    preds = gbm.predict(valid_x)
    accuracy = mean_squared_error(valid_y, preds, squared=True)
    clear_output(wait=True)
    return accuracy

In [81]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-02-26 20:40:59,201] Trial 4 finished with value: 0.23506901527135607 and parameters: {'lambda_l1': 2.22789809870267e-10, 'lambda_l2': 0.12135208552548793, 'num_leaves': 978, 'feature_fraction': 0.87, 'bagging_fraction': 0.9400000000000001, 'bagging_freq': 13, 'min_child_samples': 216}. Best is trial 0 with value: 0.23506901527135607.


Number of finished trials: 5
Best trial: {'lambda_l1': 9.951994166142383e-09, 'lambda_l2': 7.504843199605542e-09, 'num_leaves': 508, 'feature_fraction': 0.98, 'bagging_fraction': 0.77, 'bagging_freq': 10, 'min_child_samples': 602}


In [82]:
preds = gbm.predict(test_x)
preds

LightGBMError: The number of features in data (24) is not the same as it was in training data (30).
You can set ``predict_disable_shape_check=true`` to discard this error, but please be aware what you are doing.

In [78]:
dt_string = datetime.now().strftime("%d-%m-%Y_%H-%M-%S")
submission = pd.DataFrame({'id':test.id,
          'target':preds})
submission.to_csv(f'..\\kaggle_data\\{dt_string}_sub.csv', index=False)

ValueError: array length 143 does not match index length 200000